In [119]:
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

import pyperclip
import time
import datetime

def top_to_bottom(browser, i):
    start = time.time()
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(1.5)

        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height  
        if i:
            if (time.time() - start) > i:
                break
                
def print_csv(f_name, name, contact, city):
    try:
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})
        df.isnull().sum()
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df['출생연도'] = int_values
        df.columns = ['번호', '이름', '연락처', '지역', '출생연도']
        df['연락처'] = df['연락처'].str.split('\+82').str.get(1).str[0:13].fillna('없음')
        df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
        int_numbers = []
        for i in df['연락처']:
            if "10" in i:
                ii = "0" + i[1:]
                int_numbers.append(ii)
            else:
                int_numbers.append('없음')
        df['연락처'] = int_numbers
        df.to_csv(f"{f_name}_utf-8.csv", encoding='UTF-8')
    except:
        print(len(name)+' '+len(contact)+' '+len(city))
#         df.to_csv(f"{f_name}_cp949.csv", encoding='CP949')


options = webdriver.ChromeOptions()
options.add_argument("--proxy-server=socks5://127.0.0.1:9150")
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')
options.add_argument('incognito')

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)

#open browser
browser.get("https://www.facebook.com/")
time.sleep(2)

#log info, search keyword
USER = "shm8485@gmail.com"
PWD = "sohee8485"

#login
elem_id = browser.find_element("id", "email")
pyperclip.copy(USER)
elem_id.send_keys(Keys.CONTROL, "v")
time.sleep(2)

elem_pw = browser.find_element("id", "pass")
pyperclip.copy(PWD)
elem_pw.send_keys(Keys.CONTROL, "v")
elem_pw.send_keys("\n")
time.sleep(3)

#search
keyword = '음식'
kw_city = None
browser.get(f"https://www.facebook.com/search/groups/?q={keyword}")
time.sleep(2)

try:
    elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
    elem_block.click()
except:
    pass
try:
    elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
    elem_block.click()
except:
    pass
time.sleep(1)


#show only public groups
elem_public = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[2]/div/div[2]/div/input')
elem_public.click()

#city
if kw_city:
    elem_city = browser.find_element("xpath", "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[1]/div/div/div/div/div/div")
    time.sleep(1)
    elem_city.click()
    elem_city = browser.find_element("xpath", '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[1]/div/div/div/div/div/div/input')
    time.sleep(1)
    pyperclip.copy(kw_city)
    elem_city.send_keys(Keys.CONTROL, "v")
    time.sleep(1)         
    elem_city = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[2]/div/div/div[1]/div[1]/div/ul/li[1]/div/div[1]/div/div/div')
    elem_city.click()
    time.sleep(1) 

# top_to_bottom(browser, 5)
time.sleep(3)

soup_groups = bs(browser.page_source, "html.parser")
top_elements = soup_groups.find_all(class_="x1yztbdb")
groups_href = []

try:
    for i in top_elements:
        groups_href.append(i.find("a").attrs['href'])
except:
    for i in top_elements:
        print(i)

final_href = []

for i in groups_href:
#     print(i)
    final_href.append(i[:len(i)-11]+'members')
# print(len(final_href))


# full data
group_name, f_name = [], []
df_name, df_contact, df_city = [], [], []


for i in final_href[1:]:
    browser.get(i)
    try:
        elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
        elem_block.click()
    except:
        pass
    try:
        elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
        elem_block.click()
    except:
        pass
    time.sleep(1)
    # scroll to bottom
    top_to_bottom(browser, 5)
    time.sleep(1.5)
    
    soup_profile = bs(browser.page_source, "html.parser")
    g_name = soup_profile.find('h1')
    group_name.append(g_name.text)
    top_elements = soup_profile.find_all(class_="x1lq5wgf xgqcy7u x30kzoy x9jhf4c x1lliihq")
    profile_list = []
    profile_href = []

    for i in top_elements:
        profile_list.append(i.find("a").attrs['href'])

    for i in profile_list:
        profile_href.append('https://www.facebook.com/profile.php?id='+i[i.index('user/')+5:-1])

    # get real time
    timestamp = datetime.datetime.now()
    t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
    if kw_city:
        t_filename = f'{keyword}_{kw_city}_{t_stamp}'
    else:
        t_filename = f'{keyword}_{t_stamp}'
        
        
    name, contact, city = [], [], []
    error = []
    url = 'php?id='

    for i in profile_href:
        try:
            browser.get(i)
            temp = browser.current_url
            if url in temp:
                link = i + '&sk='
            else:
                link = temp[:temp.index('?')]
            browser.get(link+"about_contact_and_basic_info")
            soup_info = bs(browser.page_source, "html.parser")
            soup_name = soup_info.find(class_="x78zum5 x15sbx0n x5oxk1f x1jxijyj xym1h4x xuy2c7u x1ltux0g xc9uqle")
            name.append(soup_name.find("h1").text)
            soup_info_contact = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
            contact.append(soup_info_contact.text)
            browser.get(link+"about_places")
            soup_info = bs(browser.page_source, "html.parser")
            soup_info_city = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
            city.append(soup_info_city.text)

        except:
            l_name = len(name)
            l_cont = len(contact)
            l_city = len(city)
            print(browser.current_url)
            error.append(browser.current_url)
            if l_name > l_cont:
                contact.append('empty')
                try:
                    browser.get(link+"about_places")
                    time.sleep(1)
                    soup_info = bs(browser.page_source, "html.parser")
                    soup_info_ct = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                    city.append(soup_info_ct.text)
                except:
                    city.append('')
            elif l_cont > l_city:
                city.append('')
            else: 
                continue
    
    print_csv(t_filename, name, contact, city)
    f_name.append(t_filename)
    
timestamp = datetime.datetime.now()
t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
if kw_city:
    t_filename = f'{keyword}_{kw_city}_filelist_{t_stamp}'
else:
    t_filename = f'{keyword}_filelist_{t_stamp}'
    

df = pd.DataFrame({"그룹 이름":group_name,
                   "파일명":f_name})
df.to_csv(f"{t_filename}_utf-8.csv", encoding='UTF-8')


/groups/692297795102937/user/100047916526459/
/groups/692297795102937/user/100001068518170/
/groups/692297795102937/user/100036605537307/
/groups/692297795102937/user/100082070455422/
/groups/692297795102937/user/100028313540178/
/groups/692297795102937/user/100082070455422/
/groups/692297795102937/user/100004697315670/
/groups/692297795102937/user/100003333852487/
/groups/692297795102937/user/100092987850327/
/groups/692297795102937/user/100092977249704/
/groups/692297795102937/user/100091441599802/
/groups/692297795102937/user/100075068476374/
/groups/692297795102937/user/100078004551704/
/groups/692297795102937/user/100064935193116/
/groups/692297795102937/user/100001665407818/
/groups/692297795102937/user/100006599139134/
/groups/692297795102937/user/100087277997876/
/groups/692297795102937/user/100024108892525/
/groups/692297795102937/user/100003159596687/
/groups/692297795102937/user/100014434780851/
/groups/692297795102937/user/100001007223218/
/groups/692297795102937/user/10005

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0082A813+48355]
	(No symbol) [0x007BC4B1]
	(No symbol) [0x006C5358]
	(No symbol) [0x006AD293]
	(No symbol) [0x0070E37B]
	(No symbol) [0x0071C473]
	(No symbol) [0x0070A536]
	(No symbol) [0x006E82DC]
	(No symbol) [0x006E93DD]
	GetHandleVerifier [0x00A8AABD+2539405]
	GetHandleVerifier [0x00ACA78F+2800735]
	GetHandleVerifier [0x00AC456C+2775612]
	GetHandleVerifier [0x008B51E0+616112]
	(No symbol) [0x007C5F8C]
	(No symbol) [0x007C2328]
	(No symbol) [0x007C240B]
	(No symbol) [0x007B4FF7]
	BaseThreadInitThunk [0x770F00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77D67B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77D67B1E+238]


In [109]:
group_name

['맛있는 음식-Tasty food']

In [113]:
    for i in profile_list:
        print(i[i.index('user/')+5:-1])

100093420544472
100020756311562
106954652410354
100754726371997
106954652410354
100093420544472
100754726371997
106954652410354
100020756311562


In [96]:
a = soup2.find_all('h1')

In [99]:
print(a[1].text)

서울 부동산 /  원룸 / 자취방 / 전세 / 월세


In [88]:
print(soup2)

<html class="_9dls __fb-light-mode" dir="ltr" id="facebook" lang="ko"><head><link data-badged-icon="https://static.xx.fbcdn.net/rsrc.php/ye/r/Ta8_J_nYekI.ico" data-default-icon="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" href="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" rel="shortcut icon"/><meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/><link crossorigin="use-credentials" href="/data/manifest/" id="MANIFEST_LINK" rel="manifest"/><meta content="light" name="color-scheme"/><meta content="#FFFFFF" name="theme-color"/><style nonce="">:root{--fds-black:#000000;--fds-black-alpha-05:rgba(0, 0, 0, 0.05);--fds-black-alpha-10:rgba(0, 0, 0, 0.1);--fds-black-alpha-15:rgba(0, 0, 0, 0.15);--fds-black-alpha-20:rgba(0, 0, 0, 0.2);--fds-black-alpha-30:rgba(0, 0, 0, 0.3);--fds-black-alpha-40:rgba(0, 0, 0, 0.4);--fds-black-alpha-50:rgba(0, 0, 0, 0.5);--fds-black-alpha-60:rgba(0, 0, 0, 0.6);--fds-black-alpha-80:rgba(0, 0,

In [101]:
def print_csv(f_name, name, contact, city):
    try:
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})
        df.to_csv(f"{f_name}_utf-8.csv", encoding='UTF-8')
    except:
        print(len(name)+' '+len(contact)+' '+len(city))
#         df.to_csv(f"{f_name}_cp949.csv", encoding='CP949')

In [91]:
def check_who_has_numb(f_name):
    #번호를 가진 유저 수 확인
    df = pd.read_csv(f'{f_name}.csv', sep=',')
    df2 = df[df['연락처'].str.contains("\+82" , na=False)]
    print(df2)

In [92]:
def main(f_name, keyword=None, city=None):
    start = time.time()
    if keyword == None:
        keyword = f_name
    #crawling(file_name, keyword, city)
    links, name, contact, city, file_name = crawling(f_name, keyword, city)
    print("소요 시간: ", time.time() - start)
    
    
    return links, file_name

In [173]:
if __name__ == "__main__":
    links = main('','부동산')

UnicodeEncodeError: 'cp949' codec can't encode character '\u1ec5' in position 7: illegal multibyte sequence

In [117]:
links, f_name = main('부동산', '부동산', 'seoul')

https://www.facebook.com/groups/1513528458954109/?__tn__=%3C
https://www.facebook.com/groups/seoulrealty/?__tn__=%3C
https://www.facebook.com/groups/575386749142967/?__tn__=%3C
https://www.facebook.com/groups/161421933910183/?__tn__=%3C
https://www.facebook.com/groups/1128887303789381/?__tn__=%3C
https://www.facebook.com/groups/kyounggi/?__tn__=%3C
https://www.facebook.com/groups/248875615574618/?__tn__=%3C
https://www.facebook.com/groups/905507543268022/?__tn__=%3C
https://www.facebook.com/groups/725451924329131/?__tn__=%3C
https://www.facebook.com/groups/453667398384713/?__tn__=%3C
https://www.facebook.com/groups/148673291934658/?__tn__=%3C
https://www.facebook.com/groups/354189351728026/?__tn__=%3C
https://www.facebook.com/groups/914547538933816/?__tn__=%3C
https://www.facebook.com/groups/1983245695289827/?__tn__=%3C
https://www.facebook.com/groups/renetworkgroup/?__tn__=%3C
https://www.facebook.com/groups/337187450092574/?__tn__=%3C
https://www.facebook.com/groups/330732307760715/?

ValueError: not enough values to unpack (expected 4, got 3)